## KE-WIN
1. DCC 설치
2. KE-WIN 실행
3. DCA 실행확인
4. I/M설정, SA추가하기
5. 라이선스 코드로 받고 등록.
6. KMS연동, agent관리에서 허용아이피등록 (agtent-service 매핑되어있어야함)
7. 폴더, 접근제어 정책 생성

암호화폴더생성 버튼 누르심


## DE-MYQ
1. 환경변수 설정 / selinux설정
2. 설치파일 755 권한주기
3. 설치파일 실행 후 `ldconfig`로 연결확인
   - Idconfig -p | grep libcis_cc-3.3.so
   - Idconfig -p | grep libcis_ce-3.3.so
   - Idconfig -p | grep liblogw-${version}.so
   - Idconfig -p | grep libdamo.so
4. `install check` 실행
5. `keygen` , `keymanager`, `damo_lic` 처리
   - keygen -a -L 로 생성가능 pass입력해야함
   - keymanager -start 옵션주고
   - `ipcs -ma` 로 확인
6. ./`install_check.sh`
7. `damo_kms.conf` 설정하기, agt키 key폴더에 옮기기
8. `damo_CLI` -start 로 정책, 권한 등등 설정가능
   - create policy POL1 aes fiv > KMS연동시에는 생성 ㄴㄴ
   - set env kms only/disable/cached
   - set env double encrypt pass/error
   - set column <dbname>"<tabname>"<colname>"<pol_id>
   - set priv enc <dbuser>"<dbname>"<tabname>"<colname>"1"1
   - set env decrypt deny error/rawdata
   - reload policy/all > 정책불러오기?
   - show policy
   - export setting
   - (DB) call mysql.damo_table_migration('haksa', 'MEMBER');
   - (DB) show create table MEMBER > engine 조회 가능

   
` ALTER TABLE `test`.`MEMBER` MODIFY JUMIN varchar(50) COLLATE latin1_swedish_ci, ENGINE = InnoDB COMMENT ''; `

#### ENC MIGRATION COMMAND - 암호화 커맨드(쿼리)
` ALTER TABLE `test`.`dba` MODIFY name varchar(152) CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci, ENGINE = DAMO_INNODB DEFAULT CHARSET=utf8mb4  COMMENT '' `
#### DEC MIGRATION COMMAND - 복호화 커맨드(쿼리)
` ALTER TABLE `test`.`dba` MODIFY name varchar(100) COLLATE utf8mb4_general_ci, ENGINE = InnoDB COMMENT ''; `


## DA-MYQ

1. 환경변수
2. 파일업로드
3. 권한
   - chown -R <user>:<group> $DA_INST_HOME
   - chmod 755 log lib* acl_cli sql/install_make.sh
4. scpdb_agent.ini 설정 - key 파일들 형식 맞춰줘야함.
5. acl_cli 암복호화권한
6. plunin_dir에 damoscpdb.so 심볼릭 걸어주어야함.
   - /etc/ld.so..conf.d/mysql_damo.conf  "libdamoscpdb.so"경로저장 후 >ldconfig
   - (create user SCP 이쯤에서 함)
7. install_make 실행 및 sql파일 실행
   - ini파일이 있는 경로를 참조하게끔, 함수가 .ini 경로를 잘 찾아가게끔 함.



01환경변수
02구성품에 권한
03심볼릭 = export LD_LIBRARY_PATH?
04scpdb_agent.ini 설정 - key 파일들 형식 맞춰줘야함.
05 acl_cli 암복호화권한
save all시 떨어지는 파일을 이중화에 옮길수도있음.
06 plunin_dir에 damoscpdb.so 심볼릭 걸어주어야함.
/etc/ld.so.conf.d/mysql_damo.conf  "damoscpdb.so"경로저장 후 >ldconfig
07install_make 실행 및 sql파일 실행
ini파일이 있는 경로를 참조하게끔, 함수가 .ini 경로를 잘 찾아가게끔 함. 
001은 mysql 에 생성되는함수
002는 dbname설정해주어야함

(유저생성 및 권한부여)
/etc/my.cnf - /home, /root 밑에 "damo_da"를 두면 my.cnf가 접근하지 못한다. 최상위로 /damo_da

my.ini?


shared library - 권한/심볼릭 문제아니면 selinux문제


## DA-TIB


#### 스냅샷 Restore 후 1. 소유권 주고 시작하기. / 2. tbdown 먼저 해두기

```sh

# 환경변수 설정
vi .bash_profile
export DA_INST_HOME=/home/tibero/da-tib
. .bash_profile

# 구성파일 압축해제
tar -xvf da-tib.tar

# 실행권한주기
cd $DA_INST_HOME
chmod 755 log backup lib64/* acl_cli sql/install_make.sh

# 암호화키경로 / 로그경로 설정
vi scpdb_agent.ini
/home/tibero/da-tib/key/AES128.SCPS
/home/tibero/da-tib/log

# 심볼릭
cd $TB_HOME/lib
ls -al $DA_INST_HOME
ln -s $DA_INST_HOME/lib64/library.so library.so
ls -lart
ldd library.so

# acl_cli 실행
set priv enc SCP"KEY1"1"1" 
```
---
```sql
-- DB설정 (tbdown 해주어야 함. tbboot 모드 normal 아님)
tbsql sys/tibero
create user scp identified by scp;
grant connect, resource, create library to scp;
conn scp/scp
-- 외부라이브러리 damoscpdb를 db에 SECURE_SCP_LIB라는 이름으로 올려줌.
create library SECURE_SCP_LIB as
'/home/tibero/da-tib/lib64/libdamoscpdb.so';
/

```
---
```sh
cd $DA_INST_HOME/sql
./install_make.sh /home/tibero/da-tib


```
---
```sql

tbsql scp/scp
start 001. inner_function.tbs.sql
-- 003.sql은 옵션 : 모든사용자에게 권한을 줄때

create table emp(
  id int,
  name varchar(50),
  jumin varchar(50)
);

insert into emp values (1, 'lee', '298234-1345793');
insert into emp values (2, 'kim', '892323-2938742');
insert into emp values (3, 'park', '120849-2938743');

update emp
set jumin=enc_str('KEY1', jumin);

update emp
set jumin=dec_str('KEY1', jumin);

```

---

## DP-ORA


#### root oracle password 미리 변경
```sh

# 환경변수 설정
vi .bash_profile
export DAMO_INST_HOME=/app/oracle/product/11.2.0/dbhome_1/damo
. .bash_profile
echo $DAMO_INST_HOME

# 서버에 파일 업로드
# DCC 키생성 후 추가해주기
tar -xvf damo.tar

#심볼릭 걸어주기
cd damo
ll
cd $ORACLE_HOME/lib
ls -al $DAMO_INST_HOME | grep lib
ln -s  $DAMO_INST_HOME/library.so library.so

ls -lart
ldd library.so

# DB 설정위해 dba접속
ss
```

```sql
# DB설정 유저 및 테이블스페이스
create user scp identified by scp;
-- 안정성/효율 위함
create tablespace JONESYS datafile '/app/oracle/oradata/orcl/damo.dbf' size 500m autoextend on next 5m;

create table emp(
  id int,
  name varchar(50),
  jumin varchar(50)
);

insert into emp values (1, 'lee', '298234-1345793');
insert into emp values (2, 'kim', '892323-2938742');
insert into emp values (3, 'park', '120849-2938743');

commit;
```

```sh
# 설치파일 실행하기
cd $DAMO_INST_HOME/_sql
sh SA_Install_UnixOra10.sh

# 설치확인하기
sqlplus damo/damo
```

```sql 
select gc from dual;
```

```sh
cd ../dca
sh dca.sh start
systemctl status firewalld (root 123qwe)
systemctl stop firewalld
cat /etc/sysconfig/selinux #disalbe확인
```
```sql
-- DCC 연결
-- DCC 인증서, 키 등록
-- I/M 설정
-- DCC 계정생성 후 접속
-- SA 추가
select instance from v$thread; --INSTANCE NAME
select name from v$database; --SERVICE NAME
-- 암호화 시연
select sec_jumin from emp_damo;
select jumin from emp;
-- JONE에게 권한을 준다음
select jumin from emp;

-- 객체 재 생성
select column_name from user_tab_columns where table_name='EMP_DAMO';
alter table emp_damo rename column name to nickname;
select column_name from user_tab_columns where table_name='EMP_DAMO';
-- DCC로는 확인이 안됨.(뷰에 반영이 되지 않음)
-- 객체 재 생성후
-- DCC로 확인이 됨.(뷰에 반영이 됨)


```


### DBMS_API 방식
사용자 정의 함수를 만들고 그 함수를 이용하여 암복호화 하는 방식
DB서버에 부하가 적고 속도가 VTI방식에 비해 빠르다.

### VTI 방식

VIEW와 TRIGGER를 이용하므로 DB서버에 부하는 있을 수 있지만 편리하고(트리거 사용으로 인한 쿼리수정X) 접근에 대한 